# FLEX-clash: how to poison a model

In this notebook, we will show how easy is to manipulate the model of a client using `flexclash`. Particularly, we show the usage of the decorator `@model_poisoned`, that allow us to easily modify the model of a client inside `FlexPool`.

In our first example, we will create a federated setup using MNIST with flex primitives and then poison the model of a client.

In [1]:
from flex.datasets import load
import tensorflow as tf

flex_dataset = load("federated_emnist", return_test=False, split="digits")

2024-02-26 15:43:48.921086: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-26 15:43:49.031968: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 15:43:49.037338: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-26 15:43:49.037354: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [2]:
from flex.pool import FlexPool
from flex.pool.primitives_tf import init_server_model_tf


# Defining the model
def get_model():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

flex_pool = FlexPool.client_server_pool(fed_dataset=flex_dataset, init_func=init_server_model_tf, model=get_model())

clients = flex_pool.clients
server = flex_pool.servers
aggregator = flex_pool.aggregators

print(f"Number of nodes in the pool {len(flex_pool)}: {len(server)} server plus {len(clients)} clients. The server is also an aggregator")

Number of nodes in the pool 3580: 1 server plus 3579 clients. The server is also an aggregator


2024-02-26 15:43:51.719143: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-02-26 15:43:51.719191: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-02-26 15:43:51.719208: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mariogmarq): /proc/driver/nvidia/version does not exist
2024-02-26 15:43:51.719408: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from flex.pool.primitives_tf import deploy_server_model_tf

#Select clients
clients_per_round=20
selected_clients_pool = flex_pool.clients.select(clients_per_round)

server.map(deploy_server_model_tf, selected_clients_pool)

In [4]:
from flex.pool.primitives_tf import train_tf

selected_clients_pool.map(train_tf, batch_size=512, epochs=1, verbose=False)

Now, just before sending the client's weights to the aggregator/server, we will randomize the weights of one random client

In [5]:
from flexclash.model import model_poisoner
from flex.model import FlexModel
import numpy as np

randomized_weights_client = selected_clients_pool.select(1)

@model_poisoner
def weight_randomizer(client_model: FlexModel):
    rand_weights = [np.random.randn(*w.shape) for w in client_model["model"].get_weights()]
    client_model["model"].set_weights(rand_weights)
    return client_model

randomized_weights_client.map(weight_randomizer)

INFO:tensorflow:Assets written to: ram://f4e9fdfb-bffc-48dc-9c7f-1ba654a1c38b/assets
